# Training an RL agent with Keras-RL2 using a GEM environment

This notebook serves as an educational introduction to the usage of Keras-RL2 using a GEM environment. Goal of this notebook is to give an understanding of what Keras-RL2 is and how to use it to train and evaluate a Reinforcement Learning agent which is able to solve a current control problem within the GEM toolbox.

## 1. Installation

Before you can start you need to make sure that you have installed both, gym-electric-motor and Keras-rl2. You can install both easily using pip:

- ```pip install gym-electric-motor```
- ```pip install keras-rl2```

Alternatively, you can install them and their latest developer version directly from GitHub (recommended) :

- https://github.com/upb-lea/gym-electric-motor
- https://github.com/wau/keras-rl2

For this notebook, the following cell will do the job:

In [ ]:
!pip install -q git+https://github.com/upb-lea/gym-electric-motor.git git+https://github.com/wau/keras-rl2.git

## 2. Setting up a GEM evironment

The basic idea behind reinforcement learning is to create a so-called agent, that should learn by itself to solve a specified task in a given environment. 
This environment gives the agent feedback on its actions and reinforces the targeted behavior.
In this notebook, the task is to train a controller for the current control of a *Permanent Magnet Synchronous Motor* (*PMSM*).
 
In the following, the used GEM-environment is briefly presented, but this notebook does not focus directly on the detailed usage of GEM. If you are new to the used environment and interested in finding out what it does and how to use it, you should take a look at the [GEM cookbook](https://colab.research.google.com/github/upb-lea/gym-electric-motor/blob/master/examples/example_notebooks/GEM_cookbook.ipynb).

To save some space in this notebook, there is a function defined in an external python file called **getting_environment.py**. If you want to know how the environment's parameters are defined you can take a look at that file. By simply calling the **get_env()** function from the external file, you can set up an environment for a *PMSM* with discrete inputs.

The basic idea of the control setup from the GEM-environment is displayed in the following figure. 

![](../../docs/plots/SCML_Overview.svg)

The agent controls the converter who converts the supply currents to the currents flowing into the motor - for the *PMSM*: $i_{sq}$ and $i_{sd}$

In the continuous case, the agent's action equals a duty cycle which will be modulated into a corresponding voltage. 

In the discrete case, the agent's actions denote switching states of the converter at the given instant. Here, only a discrete amount of options are available. In this notebook, for the PMSM the *Discrete B6 Bridge Converter* with six switches is utilized per default. This converter provides a total of eight possible actions.

![](../../docs/plots/B6.svg)

The motor schematic is the following:


![](../../docs/plots/ESBdq.svg)

And the electrical ODEs for that motor are:

<h3 align="center">

<!-- $\frac{\mathrm{d}i_{sq}}{\mathrm{d}t} = \frac{u_{sq}-pL_d\omega_{me}i_{sd}-R_si_{sq}}{L_q}$

$\frac{\mathrm{d}i_{sd}}{\mathrm{d}t} = \frac{u_{sd}-pL_q\omega_{me}i_{sq}-R_si_{sd}}{L_d}$

$\frac{\mathrm{d}\epsilon_{el}}{\mathrm{d}t} = p\omega_{me}$
 -->

   $ \frac{\mathrm{d}i_{sd}}{\mathrm{d}t}=\frac{u_{sd} + p\omega_{me}L_q i_{sq} - R_s i_{sd}}{L_d} $ <br><br>
    $\frac{\mathrm{d} i_{sq}}{\mathrm{d} t}=\frac{u_{sq} - p \omega_{me} (L_d i_{sd} + \mathit{\Psi}_p) - R_s i_{sq}}{L_q}$ <br><br>
   $\frac{\mathrm{d}\epsilon_{el}}{\mathrm{d}t} = p\omega_{me}$

</h3>

The target for the agent is now to learn to control the currents. For this, a reference generator produces a trajectory that the agent has to follow. 
Therefore, it has to learn a function (policy) from given states, references and rewards to appropriate actions.

For a deeper understanding of the used models behind the environment see the [documentation](https://upb-lea.github.io/gym-electric-motor/).
Comprehensive learning material to RL is also [freely available](https://github.com/upb-lea/reinforcement_learning_course_materials).

The following code is using the pre-written function ```get_env()``` to import a pre-defined GEM environment. 
Additionally, a ```RewardLogger``` callback is added to visualize the training progress.

In [ ]:
from getting_environment import get_env
env = get_env(training=True)

## 3. Train an Agent with Keras-rl2

[Keras-rl2](https://github.com/wau/keras-rl2) is a widely used Python library for reinforcement learning. It compiles a collection of Reinforcement Learning algorithm implementations.
Most of the algorithms and their extensions are readily available and are quite stable. The ease of adopting to various environments adds to its popularity.
For currently available RL algorithms see their [documentation](https://keras-rl.readthedocs.io/en/latest/agents/overview/)

### 3.1 Imports
The environment in this control problem poses a discrete action space. Therefore, the [Deep Q-Network (DQN)](https://arxiv.org/abs/1312.5602) is a suitable agent.

In [ ]:
from pathlib import Path
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from getting_environment import get_env

### 3.2 Parameters and Instantiation

For the DQN algorithm, a set of parameters has to be pre-defined.
In particular, a multilayer perceptron (MLP) is used as function approximator within the DQN algorithm.
Note: Although there are 8 possible actions, we ignore the last action as the voltage applied on the motor there is the same as for the first action. This effectively reduces the feature space that is to be explored and benefits training.

In [ ]:
# Training parameters.
gamma = 0.99
time_limit = True
buffer_size = 200000  # observation history size
batch_size = 25  # mini batch size sampled from history at each update step
env = get_env(time_limit, gamma, training=True)
nb_actions = env.action_space.n
window_length = 1

# construct a MLP
model = Sequential()
model.add(Flatten(input_shape=(window_length,) + env.observation_space.shape))
model.add(Dense(64, activation='relu'))  # hidden layer 1
model.add(Dense(64, activation='relu'))  # hidden layer 2
model.add(Dense(nb_actions, activation='linear'))  # output layer

# keras-rl2 objects
memory = SequentialMemory(limit=200000, window_length=window_length)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(eps=0.2), 'eps', 1, 0.05, 0, 50000)

### 3.3 Training

Once you've setup the environment and defined your parameters starting the training is nothing more than a one-liner. For each algorithm all you have to do is call its ```fit()``` function.

The advantage of keras-rl is that you can see all the important parameters and their values during training phase to get an intution of learning.
Hence, there is no need of additional callbacks.

In [ ]:
dqn = DQNAgent(
    model=model,
    policy=policy,
    nb_actions=nb_actions,
    memory=memory,
    gamma=gamma,
    batch_size=25,
    train_interval=1,
    memory_interval=1,
    target_model_update=1000,
    nb_steps_warmup=10000,
    enable_double_dqn=True
)

dqn.compile(Adam(lr=1e-4),
            metrics=['mse']
            )

In [ ]:
%matplotlib notebook
history = dqn.fit(env,
            nb_steps=500000,
            action_repetition=1,
            verbose=2,
            visualize=True,
            nb_max_episode_steps=10000,
            log_interval=1000
            )

### 3.4 Saving the model

When the training is finished, you can save the model that your DQN-agent has learned for later reuse, e.g. for evaluation or continued training. For this purpose, ```save_weights()``` is used.

In [ ]:
weight_path = Path('saved_agents')
weight_path.mkdir(parents=True, exist_ok=True)
dqn.save_weights(str(weight_path / 'dqn_keras-RL2.hdf5'), overwrite=True)

## 5. Evaluating an agent

Keras-rl2 agents have their own ```test()``` method which can be used to evaluate their performance.
Here, the metric used is the total reward the agent accumulates at the end of each episode. 

### 5.1 Loading a Model (Optional)

In [ ]:
dqn.load_weights('saved_agents\dqn_keras-RL2.hdf5')

### 5.2 Testing

In [ ]:
%matplotlib notebook
env = get_env(time_limit = False, gamma=0.99, training=False)
dqn.test(env,
         nb_episodes=5,
         nb_max_episode_steps=100000,
         visualize=True
         )